In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import matplotlib.patches as patches

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from torchvision import transforms

files_dir = '../input/wadhwani-ai-bollworm/images_resized/images_resized'

In [2]:
sample_sub = pd.read_csv('../input/wadhwani-ai-bollworm/SampleSubmission.csv')
sample_sub.head()

,image_id_worm,number_of_worms
0,id_00332970f80fa9a47a39516d_abw,0
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,0
4,id_005102f664b820f778291dee_abw,0


In [3]:
sample_sub.shape

(5606, 2)

In [4]:
test = pd.read_csv('../input/wadhwani-ai-bollworm/Test.csv')
test.head()

,image_id_worm
0,id_00332970f80fa9a47a39516d.jpg
1,id_0035981bc3ae42eb5b57a317.jpg
2,id_005102f664b820f778291dee.jpg
3,id_0066456f5fb2cd858c69ab39.jpg
4,id_007159c1fa015ba6f394deeb.jpg


In [5]:
test.shape

(2803, 1)

# Test dataset 

In [6]:
class BollwormTestDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id_worm'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id_worm'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        if self.transforms is not None:  #Apply transformation
            image = self.transforms(image)

        '''if self.transforms:
            sample = {
                'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']'''

        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [7]:
# Send train=True fro training transforms and False for val/test transforms
trans = transforms.Compose([transforms.ToTensor()])

# Load model on our saved weights

In [8]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

In [9]:
WEIGHTS_FILE = '../input/wadhwani-ai-bollworm/fasterrcnn_resnet50_fpn.pth'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 3  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.eval()

x = model.to(device)

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = BollwormTestDataset(test, files_dir,trans)

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    collate_fn=collate_fn
)

In [11]:
# the function takes the original prediction and the iou threshold.

def apply_nms(orig_prediction, iou_thresh=0.3):
    
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [16]:
#results = []

#for images, image_ids in test_data_loader:

    #images = list(image.to(device) for image in images)
    #outputs = model(images)

    #for i, image in enumerate(images):

        
        #labels = outputs[i]['labels'].data.cpu().numpy()
        #image_id = image_ids[i]
        
        ## new_values_ib = {'pbw': 1, 'abw': 2}
        #sum1 =0
        #sum2 =0
        #for it in labels:
            #if it == 1:
                #sum1+=1
            #else:
                #sum2+=1
                
        #result = {
            #'image_id': image_id,
            #'pbw': sum1,
            #'abw':sum2
        #}

        
        #results.append(result)

# Create the submission file 

In [12]:
new_results = []

for images, image_ids in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)
    cpu_device = torch.device("cpu")
    outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
    
    for i, image in enumerate(images):
        
        
        nms_outputs = apply_nms(outputs[i], iou_thresh=0.2)
        labels = nms_outputs['labels'].numpy()
        image_id = image_ids[i]
        
        #new_values_ib = {'pbw': 1, 'abw': 2}
        sum1 =0
        sum2 =0
        for it in labels:
            if it == 1:
                sum1+=1
            else:
                sum2+=1
                
        result = {
            'image_id': image_id,
            'pbw': sum1,
            'abw':sum2
        }

        
        new_results.append(result)

In [14]:
new_results[6:10]

[{'image_id': 'id_00887bebda26184c36e18e00.jpg', 'pbw': 13, 'abw': 1},
 {'image_id': 'id_00ba116c0f45a71a7e0e652c.jpg', 'pbw': 0, 'abw': 0},
 {'image_id': 'id_00c11f7689e2351305cb12e3.jpg', 'pbw': 19, 'abw': 0},
 {'image_id': 'id_00c1c4a5f0e0354f8e2aa416.jpg', 'pbw': 1, 'abw': 4}]

In [15]:
new_one = pd.DataFrame(new_results, columns=['image_id', 'pbw', 'abw'])
new_one.head()

,image_id,pbw,abw
0,id_00332970f80fa9a47a39516d.jpg,0,4
1,id_0035981bc3ae42eb5b57a317.jpg,23,0
2,id_005102f664b820f778291dee.jpg,0,9
3,id_0066456f5fb2cd858c69ab39.jpg,0,5
4,id_007159c1fa015ba6f394deeb.jpg,0,1


In [16]:
newdf = pd.DataFrame(np.repeat(new_one.values, 2, axis=0), columns=new_one.columns)
newdf.head()

,image_id,pbw,abw
0,id_00332970f80fa9a47a39516d.jpg,0,4
1,id_00332970f80fa9a47a39516d.jpg,0,4
2,id_0035981bc3ae42eb5b57a317.jpg,23,0
3,id_0035981bc3ae42eb5b57a317.jpg,23,0
4,id_005102f664b820f778291dee.jpg,0,9


In [17]:
newdf['image_id'] = sample_sub['image_id_worm']

In [18]:
newdf.head()

,image_id,pbw,abw
0,id_00332970f80fa9a47a39516d_abw,0,4
1,id_00332970f80fa9a47a39516d_pbw,0,4
2,id_0035981bc3ae42eb5b57a317_abw,23,0
3,id_0035981bc3ae42eb5b57a317_pbw,23,0
4,id_005102f664b820f778291dee_abw,0,9


In [19]:
def return_num(data):
    image = data.image_id
    label_1 = data.pbw
    label_2 = data.abw
    
    if image[-3:] == 'abw':
        return label_2
    else:
        return label_1

In [20]:
newdf['label'] = newdf.apply(return_num, axis=1)

In [21]:
newdf.head()

,image_id,pbw,abw,label
0,id_00332970f80fa9a47a39516d_abw,0,4,4
1,id_00332970f80fa9a47a39516d_pbw,0,4,0
2,id_0035981bc3ae42eb5b57a317_abw,23,0,0
3,id_0035981bc3ae42eb5b57a317_pbw,23,0,23
4,id_005102f664b820f778291dee_abw,0,9,9


In [22]:
sample_sub['number_of_worms'] = newdf['label']

In [23]:
sample_sub.head()

,image_id_worm,number_of_worms
0,id_00332970f80fa9a47a39516d_abw,4
1,id_00332970f80fa9a47a39516d_pbw,0
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,23
4,id_005102f664b820f778291dee_abw,9


In [25]:
sample_sub.to_csv('sampble_sub_with_nms.csv', index=False)